## Configurações iniciais

## Gerando população inicial de cromossomos

In [6]:
import random

class Cromossome:
    def __init__(self, genetics=None):
        self.length = 20
        self.mutation_rate = 0.01
        self.generate_cromossome(genetics)
        self.score = self.fitness()
       
    def __str__(self):
        #return str(self.cromossome)
        result = ''
        for bit in self.cromossome:
            result += '1' if bit else '0'
        return result
    
    def generate_cromossome(self, genetics):
        if genetics:
            self.cromossome = genetics
        else:
            self.cromossome = []
            for i in range(self.length):
                self.cromossome.append(random.randint(0,1) == 1)
    
    def mutation(self):
        for i in range(self.length):
            if random.random() < self.mutation_rate:
                self.cromossome[i] = type(self.cromossome[i])(not self.cromossome[i])
            
    def fitness(self):
        OPTIMAL = 20*'1'
        fitness = 0
        for c in range(self.length):
            fitness += abs(ord(str(self)[c]) - ord(OPTIMAL[c]))
        return self.length - fitness

In [7]:
import operator

class Population:
    def __init__(self):
        self.size = 30
        self.crossover_rate = 0.7
        self.individuals = []
        self.score = self.calculate_score()
        self.history = []
        
    def add(self, individual):
        self.individuals.append(individual)
        self.score = self.calculate_score()
        
    def calculate_score(self):
        self.score = 0
        for individual in self.individuals:
            self.score += individual.fitness()
        return self.score
    
    def crossover(self):
        for i in range(len(self.individuals)):
            if random.random() < self.crossover_rate:
                cromossome_A, cromossome_B = 0, 0
                while cromossome_A == cromossome_B:
                    cromossome_A, cromossome_B = random.choice(self.individuals), random.choice(self.individuals)
                pos = int(random.random()*cromossome_A.length)
                self.sort()
                self.individuals.pop()
                self.individuals.pop()
                self.add(Cromossome(genetics=cromossome_A.cromossome[:pos]+cromossome_B.cromossome[pos:]))
                self.add(Cromossome(genetics=cromossome_B.cromossome[:pos]+cromossome_A.cromossome[pos:]))
    
    def sort(self):
        self.individuals.sort(key=operator.attrgetter('score'), reverse=True)
        
    def drop(self, vitims):
        for i in range(vitims):
            self.individuals.pop()

In [8]:
import pika
import json

credentials = pika.PlainCredentials('guest', 'guest')
connection = pika.ConnectionParameters('rabbitmq',
                                       5672,
                                       '/',
                                       credentials)

connection = pika.BlockingConnection(connection)
channel = connection.channel()

channel.queue_declare(queue='genetico')


<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=genetico'])>"])>

In [9]:
RUN_NUMBER = 1

EXPERIMENT_NAME = 'Exp_001'

for i in range(20):
    population = Population()
    for x in range(population.size):
        population.add(Cromossome())
    '''
    population.history.append(population.score)
    print('Population Score:', population.score)

    print ('Cromossome\t\tFitness')
    population.sort()
    for cromo in population.individuals:
        print(str(cromo) + '\t' + str(cromo.fitness()))
    '''

    for j in range(10):
        population.crossover()

        for individual in population.individuals:
            individual.mutation()

        #print('Population Score:', population.score)
        population.history.append(population.score)

        #print ('\nCromossome\t\tFitness')
        population.sort()

        for cromo in population.individuals:
            message = {'experiment': EXPERIMENT_NAME,
               'run': i,
               'generation': j,
               'variables': cromo.cromossome,
               'fitness': cromo.fitness() }
            #print(message)
            channel.basic_publish(exchange='',
                                  routing_key='genetico',
                                  body=json.dumps(message))

    print(population.history)

[333, 397, 478, 519, 550, 572, 574, 597, 592, 587]
[389, 446, 495, 524, 540, 549, 570, 599, 592, 580]
[371, 423, 493, 525, 549, 571, 575, 596, 590, 582]
[403, 469, 490, 520, 542, 578, 591, 591, 583, 576]
[424, 464, 491, 527, 563, 591, 596, 590, 585, 576]
[389, 451, 502, 526, 551, 568, 562, 555, 550, 545]
[348, 387, 408, 440, 472, 509, 551, 567, 563, 562]
[405, 462, 483, 491, 511, 511, 527, 537, 536, 556]
[427, 489, 522, 547, 574, 583, 596, 589, 584, 577]
[368, 406, 447, 473, 522, 548, 564, 560, 557, 550]
[411, 457, 483, 511, 521, 539, 550, 577, 597, 587]
[347, 412, 459, 493, 529, 547, 554, 577, 600, 596]
[394, 451, 491, 512, 524, 543, 562, 574, 591, 594]
[370, 459, 502, 526, 535, 549, 569, 563, 553, 545]
[382, 442, 493, 528, 555, 572, 594, 592, 584, 578]
[405, 459, 476, 503, 546, 564, 581, 596, 594, 591]
[371, 405, 445, 481, 510, 499, 509, 539, 553, 567]
[385, 450, 492, 519, 534, 548, 569, 570, 574, 595]
[363, 421, 477, 518, 528, 544, 568, 563, 559, 554]
[384, 430, 456, 478, 512, 537, 

In [5]:
connection.close()